In [1]:
# Importamos las librerías.
from bs4 import BeautifulSoup
import requests

# Scraping

In [2]:
# 1. Obtenemos el HTML
URL_BASE = 'https://scrapepark.org/courses/spanish/'
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

# 2. "Parsear" ese HTML
soup = BeautifulSoup(html_obtenido, "html.parser")

### El método **find()**

Permite quedarnos con la información asociada a una etiqueta de HTML

In [3]:
primer_h2 = soup.find('h2')
print(primer_h2)

<h2>¿Por qué comprar con nosotros?</h2>


In [4]:
# Solo el texto.
print(primer_h2.text)

# Equivalente a: 
print(soup.h2.text)

¿Por qué comprar con nosotros?
¿Por qué comprar con nosotros?


### El método **find()**

Busca **TODOS** los elementos de la página con esa etiqueta y devuelve una "lista" que los contiene (en realidad devuelve un objeto de la clase bs4.element.ResultSet).

In [5]:
h2_todos = soup.find_all('h2')
print(h2_todos)

[<h2>¿Por qué comprar con nosotros?</h2>, <h2>
                  #Novedades
                </h2>, <h2>
            Nuestros <span>productos</span>
</h2>, <h2>
            Testimonios de clientes
          </h2>, <h2 class="heading-container">
          Tabla de precios
        </h2>]


In [6]:
# Si usamos el parámetro limit = 1, emulamos al metodo find
h2_uno_solo = soup.find_all('h2', limit = 1)
print(h2_uno_solo)

[<h2>¿Por qué comprar con nosotros?</h2>]


In [7]:
# Podemos iterar sobre el objeto.
for seccion in h2_todos:
    print(seccion.text)

¿Por qué comprar con nosotros?

                  #Novedades
                

            Nuestros productos


            Testimonios de clientes
          

          Tabla de precios
        


In [8]:
# get_text() para más funcionalidades.
for seccion in h2_todos:
    print(seccion.get_text(strip = True)) # Limpiamos los espacios en blanco.

¿Por qué comprar con nosotros?
#Novedades
Nuestrosproductos
Testimonios de clientes
Tabla de precios


### Utilizando atributos de la página.

In [9]:
# Clase.
divs = soup.find_all('div', class_ = "heading-container heading-center")

for div in divs:
    print(div)
    print('\n')

<div class="heading-container heading-center" id="acerca">
<h2>¿Por qué comprar con nosotros?</h2>
</div>


<div class="heading-container heading-center" id="productos">
<h2>
            Nuestros <span>productos</span>
</h2>
</div>


<div class="heading-container heading-center">
<h3>Suscríbete para obtener descuentos y ofertas</h3>
</div>


<div class="heading-container heading-center">
<h2>
            Testimonios de clientes
          </h2>
</div>




In [10]:
# Todas las etiquetas que tengan el atributo "src".
src_todos = soup.find_all(src = True)

for elemento in src_todos:
    if elemento['src'].endswith('.jpg'):
        print(elemento)

<img alt="Parque de patinaje" src="images/slider-bg.jpg"/>
<img alt="Patineta 2" src="images/p2.jpg"/>


Bajamos todas las imágenes.

In [11]:
url_imagenes = []

for i, imagen in enumerate(src_todos):
    if imagen['src'].endswith('png'):
        print(imagen['src'])
        r = requests.get(f'https://scrapepark.org/courses/spanish/{imagen['src']}')

        with open(f'imagen_{i}.png', 'wb') as f:
            f.write(r.content)

images/arrival-bg-store.png
images/p1.png
images/p3.png
images/p4.png
images/p5.png
images/p6.png
images/p7.png
images/p8.png
images/p9.png
images/p10.png
images/p11.png
images/p12.png
images/client-one.png
images/client-two.png
images/client-three.png
./images/freecodecamp-logo.png


In [12]:
# Información de tablas.
URL_BASE = 'https://scrapepark.org/courses/spanish'
URL_TABLA = soup.find_all('iframe')[0]['src']

requests_tabla = requests.get(f'{URL_BASE}/{URL_TABLA}')

html_tabla = requests_tabla.text
soup_tabla = BeautifulSoup(html_tabla, 'html.parser')
soup_tabla.find('tabla')

productos_faltantes = soup_tabla.find_all(['th', 'td'], attrs = {'style': 'color: red;'})
productos_faltantes = [talle.text for talle in productos_faltantes]

print(productos_faltantes)

['Longboard', '$80', '$85', '$90', '$62', '$150']


In [13]:
divs = soup.find_all('div', class_ = 'detail-box')

nombre_producto = []
precio_producto = []

for div in divs:
    if (div.h6 is not None) and ('Patineta' in div.h5.text):
        producto = div.h5.get_text(strip = True)
        precio = div.h6.get_text(strip = True).replace('$', '')

        print(f'Producto: {producto:<17} | Precio: {precio}')
        nombre_producto.append(producto)
        precio_producto.append(precio)

Producto: Patineta Nueva 1  | Precio: 75
Producto: Patineta Usada 2  | Precio: 80
Producto: Patineta Nueva 3  | Precio: 68
Producto: Patineta Usada 4  | Precio: 70
Producto: Patineta Nueva 5  | Precio: 75
Producto: Patineta Nueva 6  | Precio: 58
Producto: Patineta Nueva 7  | Precio: 80
Producto: Patineta Nueva 8  | Precio: 35
Producto: Patineta Nueva 9  | Precio: 165
Producto: Patineta Usada 10 | Precio: 54
Producto: Patineta Usada 11 | Precio: 99
Producto: Patineta Nueva 12 | Precio: 110


Cambios que dependen de la URL

In [14]:
URL_BASE = 'https://scrapepark.org/courses/spanish/contact'

for i in range(1, 3):
    URL_FINAL = f"{URL_BASE}{i}"
    print(URL_FINAL)
    r = requests.get(URL_FINAL)
    soup = BeautifulSoup(r.text, "html.parser")
    print(soup.h5.text)

https://scrapepark.org/courses/spanish/contact1
Texto que cambia entre páginas en contacto 1 :)
https://scrapepark.org/courses/spanish/contact2
Texto que cambia entre páginas en contacto 2 :)


Datos que no sabemos en que parte de la página se encuentran.

In [15]:
# Expresiones regulares.
import re

# Obtenemos el HTML
URL_BASE = 'https://scrapepark.org/courses/spanish'
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

# Parseamos el HTML
soup = BeautifulSoup(html_obtenido, "html.parser")

telefonos = soup.find_all(string = re.compile(r"\d+-\d+-\d+"))
telefonos

[' 4-444-4444']

Moviéndonos por el árbol.

In [16]:
copyrights = soup.find_all(string = re.compile('©'))
copyrights[0]

'© 2022 '

In [17]:
primer_copyright = copyrights[0]
primer_copyright.parent

<p>© 2022 <span>Todos los derechos reservados</span>.
        <a href="https://html.design/" rel="noopener noreferrer" target="_blank">Creado con Free Html Templates</a>.
      </p>

In [18]:
menu = soup.find(string = re.compile('MENÚ'))
menu.parent.find_next_siblings()

[<ul>
 <li><a href="#">Inicio</a></li>
 <li><a href="#">Acerca</a></li>
 <li><a href="#">Servicios</a></li>
 <li><a href="#">Testimonios</a></li>
 <li><a href="#">Contacto</a></li>
 </ul>]

Comentario sobre excepciones.

In [19]:
strings_a_buscar = ['MENÚ', '©', 'Helado', 'Patineta']

for string in strings_a_buscar:

    try:
        resultado = soup.find(string = re.compile(string))
        print(resultado.text)

    except AttributeError:
        print(f'El string "{string}" no fue encontrado')

MENÚ
© 2022 
El string "Helado" no fue encontrado

                  Patineta Nueva 1
                


Guardamos los datos.

In [20]:
nombre_producto.insert(0, 'productos')
precio_producto.insert(0, 'precios')

datos = dict(zip(nombre_producto, precio_producto))

In [21]:
import csv

with open('datos.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(datos.items())

**Ejercicios**:

1. Las patinetas que valgan menos que $68
2. Las patinetas que en su nombre tengan un número superior a 3
3. Traer cualquier texto de la página que tenga la palabra descuento u oferta.
4. Gnerar un archivo .csv con dos columnas: Una columna que contenga el nombre del cliente y la otra columna que contenga su reseña.

In [22]:
URL_BASE = 'https://scrapepark.org/courses/spanish/'
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

soup = BeautifulSoup(html_obtenido, "html.parser")

In [35]:
# 1. Obtenemos las patinetas que valgan menos que $68

nombre_patineta = []
precio_patineta = []

for div in divs:
    if (div.h6 is not None) and ('Patineta' in div.h5.text):
        patineta = div.h5.get_text(strip = True)
        precio = div.h6.get_text(strip = True).replace('$', '')

        precio = int(precio)
        

68
58
35
54


In [30]:
print(f'Producto: {nombre_patineta:<17} | Precio: {precio_patineta}')

TypeError: unsupported format string passed to list.__format__